In [ ]:
import numpy as np
import plotly.express as px
import os
import nibabel as nib
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import math
from sklearn.linear_model import LinearRegression
from dash import Dash, dcc, html
import pandas as pd
PI_UNICODE = "\U0001D70B"
GYRO_BAR_RATIO_H = 42.6e6  # [Hz/T]

In [ ]:
# Taken from real brain data
phase1, phase2, phase3, phase4 = (2.9298516102709202, -0.21864564255753116, -2.2884910587688285, 2.392827225041896)
beg = 0
end = 0.015

t = np.array([0.00263, 0.00526, 0.009, 0.013])
n_echoes = len(t)
y = np.array([phase1, phase2, phase3, phase4])
y_unwrapped = np.array([phase1, phase2, phase3, phase4 - (2 * math.pi)])

reg = LinearRegression().fit(t.reshape(-1, 1), y_unwrapped.reshape(-1,1))
# Slope of linear regression reshaped into the shape of original 3D phase.
fieldmap_rad = reg.coef_[0]  # [rad / s]
fieldmap_intercept = reg.intercept_[0]  # [rad / s]

t_predict = np.array([beg, end])
y_predict = reg.predict(t_predict.reshape(-1,1))[:,0]


fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=y, mode='markers', marker=dict(color='blue'), name='Original Phase'))
fig.add_trace(go.Scatter(x=[t[3]], y=[phase4- (2*math.pi)], mode='markers', marker=dict(color='red'), name='Unwrapped Phase'))
fig.add_trace(go.Scatter(x=t_predict, y=y_predict, mode='lines', marker=dict(color='green'), name='Fit'))
fig.add_trace(go.Scatter(x=[beg, end], y=[math.pi, math.pi], mode='lines', line=dict(color='gray'), showlegend=False))
fig.add_trace(go.Scatter(x=[beg, end], y=[-math.pi, -math.pi], mode='lines', line=dict(color='gray'), showlegend=False))

fig.update_xaxes(title_text="Time (ms)", range=[beg, end])
fig.update_yaxes(title_text="Phase (rad)", tickmode = 'array', range=[-7,7],
                 tickvals = [-2*math.pi, -math.pi, 0, math.pi, 2*math.pi],
                 ticktext = [f'-2{PI_UNICODE}', f'-{PI_UNICODE}', '0', f'{PI_UNICODE}', f'2{PI_UNICODE}'])
fig.update_layout({"width": 800})

fig.show()

In [ ]:
t = np.linspace(0, 10, 1001)
a = 2
y = a * t - 10

# 
phase = np.angle(np.exp(1j*y))

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=phase, mode='lines', name='Wrapped phase'))
fig.add_scatter(x=t, y=y, mode='lines', name='True phase')
fig.update_traces(marker=dict(size=3))
fig.update_xaxes(title_text="x")
fig.update_yaxes(title_text="rad", tickmode = 'array',
        tickvals = [-3*math.pi, -2*math.pi, -math.pi, 0, math.pi, 2*math.pi, 3*math.pi],
        ticktext = [f'-3{PI_UNICODE}', f'-2{PI_UNICODE}', f'-{PI_UNICODE}', '0', f'{PI_UNICODE}', f'2{PI_UNICODE}', f'3{PI_UNICODE}'])
fig.update_layout({"width": 800})
fig.show()


In [ ]:
import dash
import dash_bootstrap_components as dbc
from dash import Dash, html, dcc
import plotly.express as px

t = np.linspace(0, 3, 1001)
y_unwrapped = t + t**2 - 3 * t**3 + t**4
y_wrapped = np.mod(y_unwrapped, 2 * math.pi);

y_delta = y_unwrapped.max() - y_unwrapped.min()
min_y = y_unwrapped.min() - 2 * math.pi - y_delta * 0.05
max_y = y_unwrapped.max() + 2 * math.pi + y_delta * 0.05

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=t, y=y_wrapped,
                          mode='lines',
                          name='Wrapped  '))
fig1.update_layout(title_text="Wrapped", title_x=0.25, showlegend=True,
                   legend={"x": 0.03, "y": 0.95})
fig1.update_yaxes(range=[min_y, max_y], title_text="rad", tickmode = 'array',
                  tickvals = [-2*math.pi, -math.pi, 0, math.pi, 2*math.pi, 3*math.pi, 4*math.pi, 5*math.pi],
                  ticktext = [f'-2{PI_UNICODE}', f'-{PI_UNICODE}', '0', f'{PI_UNICODE}', f'2{PI_UNICODE}', f'3{PI_UNICODE}', f'4{PI_UNICODE}', f'5{PI_UNICODE}'])
fig1.update_xaxes(fixedrange=True)
fig1.update_yaxes(fixedrange=True)

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=t, y=y_unwrapped,
                          mode='lines',
                          name='Solution 1', showlegend=True))
fig2.add_trace(go.Scatter(x=t, y=y_unwrapped+2*math.pi,
                          mode='lines',
                          name='Solution 2', showlegend=True))
fig2.add_trace(go.Scatter(x=t, y=y_unwrapped-2*math.pi,
                          mode='lines',
                          name='Solution 3', showlegend=True))
fig2.update_layout(title_text="Unwrapped", title_x=0.75,
                   legend={"x": 0.85, "y": 0.95})
fig2.update_yaxes(range=[min_y, max_y], title_text="rad", tickmode = 'array',
                  tickvals = [-2*math.pi, -math.pi, 0, math.pi, 2*math.pi, 3*math.pi, 4*math.pi, 5*math.pi],
                  ticktext = [f'-2{PI_UNICODE}', f'-{PI_UNICODE}', '0', f'{PI_UNICODE}', f'2{PI_UNICODE}', f'3{PI_UNICODE}', f'4{PI_UNICODE}', f'5{PI_UNICODE}'])
fig2.update_xaxes(fixedrange=True)
fig2.update_yaxes(fixedrange=True)
app = Dash(
    __name__,
    external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.FONT_AWESOME],
    external_scripts=[{'src':"https://ajax.googleapis.com/ajax/libs/jquery/3.6.0/jquery.min.js"}]
)

def beforeAfterSlide(fig1, fig2, style=None):
    bfA = []
    if not style:
        style = {'width':'100vw', 'height':'100vh'}
    for key in style:
        if '%' in style[key]:
            if key in ['width', 'left']:
                style[key] = style[key].replace('%','vw')
            if key in ['top', 'height']:
                style[key] = style[key].replace('%','vh')
    bfA.append(html.Div(dcc.Graph(figure=fig2, style=style), className='after'))
    bfA.append(html.Div(className='middle'))
    bfA.append(html.Div(dcc.Graph(figure=fig1, style=style), className='before'))
    return html.Div(bfA, className='beforeAfter', style=style)

app.layout = html.Div(beforeAfterSlide(fig1, fig2, {'height':'75%', 'width':'75%', 'top':'10%', 'left':'6%'}))

if __name__ == "__main__":
    app.run_server(debug=True)

In [ ]:
seed = 2
np.random.seed(seed)
n_points = 20
len_x = 500
len_y = 500
map = np.zeros([len_x, len_y]) - math.pi
x, y = np.meshgrid(range(len_x), range(len_y))

def gauss(a, std_x, std_y, center):
    x0 = int(center[0] * len_x)
    y0 = int(center[1] * len_y)
    return a * np.exp(-((((x-x0)**2)/(std_x**2)) + (((y-y0)**2)/(std_y**2))))


for i_point in range(n_points):
    point = (np.random.uniform(), np.random.uniform())
    amp = np.random.randint(4, 9)
    std_x = np.random.randint(50, 100)
    std_y = np.random.randint(50, 100)
    map += gauss(amp, std_x, std_y, point)

wrapped = np.angle(np.exp(1j*map))

fig = make_subplots(rows=1, cols=2, shared_xaxes=False, horizontal_spacing=0.1, subplot_titles=("Wrapped", "Unwrapped"), specs=[[{"type": "Heatmap"}, {"type": "Heatmap"}]])
fig.add_trace(go.Heatmap(z=wrapped, colorscale='gray', colorbar_x=0.47, colorbar=dict(title="Rad", titleside="top", tickmode="array", tickvals=[-math.pi, 0, math.pi], ticktext = [f'-{PI_UNICODE}', 0, f'{PI_UNICODE}'])), 1, 1)
fig.add_trace(go.Heatmap(z=map, colorscale='gray', colorbar=dict(title="Rad", titleside="top", tickmode="array",
                                                                 tickvals=[-2*math.pi, 0, 2*math.pi, 4*math.pi, 6*math.pi, 8*math.pi, 10*math.pi, 12*math.pi, 14*math.pi],
                                                                 ticktext = [f'-2{PI_UNICODE}', 0, f'2{PI_UNICODE}', f'4{PI_UNICODE}', f'6{PI_UNICODE}', f'8{PI_UNICODE}', f'10{PI_UNICODE}', f'12{PI_UNICODE}', f'14{PI_UNICODE}'])), 1, 2)
fig.update_layout({"height": 500, "width": 1000})
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()

In [ ]:
len_x = 500
len_y = 500
x, y = np.meshgrid(np.linspace(-len_x, len_x/2, len_x//2), np.linspace(-len_y, len_y, len_y), indexing='ij')
x1 = np.array(x)
y1 = np.array(y)
x, y = np.meshgrid(np.linspace(-len_x/2, len_x, len_x//2), np.linspace(-len_y, len_y, len_y), indexing='ij')
x2 = np.array(x)
y2 = np.array(y)
x = np.zeros([len_x, len_y])
y = np.zeros([len_x, len_y])
x[:len_x//2] = x1
y[:len_x//2,:] = -y1
x[len_x//2:,:] = -x2
y[len_x//2:,:] = -y2


phase = np.rot90(np.arctan2(y.astype(float),x.astype(float)),k=-1)
fig = make_subplots(rows=1, cols=1, subplot_titles=[("Phase Singularities")], specs=[[{"type": "Heatmap"}]])
fig.add_trace(go.Heatmap(z=phase, colorscale='gray', colorbar=dict(title="Rad", titleside="top", tickmode="array", tickvals=[-math.pi+0.01, 0, math.pi-0.01], ticktext = [f'-{PI_UNICODE}', 0, f'{PI_UNICODE}'])))
fig.add_trace(go.Scatter(x=[370, 370], y=[90, 410], mode='markers',  marker=dict(size=10), line=dict(color='red')))

fig.add_shape(type="line", x0=370, y0=90, x1=370, y1=410, line=dict(color="red",width=3))
fig.add_shape(type="path", path="M 370,90 Q 200,250 370,410", line=dict(color="red",width=3))
fig.add_annotation(x=390, y=90, text="A", showarrow=False)
fig.add_annotation(x=390, y=410, text="B", showarrow=False, font=dict(color="white"))
fig.update_layout({"width": 500, "height": 400})
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()

In [ ]:
b0 = np.array((0.064, 0.1, 0.3, 1.5, 3, 7))  # [T]
f0 = b0 * GYRO_BAR_RATIO_H  # [Hz]
df = 1e-6 * f0
te = 1e3/(2*df)

tmp = {
    'B0': list(b0),
    'TE (ms)': list(te)
}
df = pd.DataFrame(tmp)
# df.to_markdown()
df = df.T.style.hide(axis='columns')